# Deep SRCNN (residual blocks)

In [21]:
from data import DIV2K
train = DIV2K(scale=4, downgrade='bicubic', subset='train')
train_ds = train.dataset(batch_size=16, random_transform=True)

In [22]:
# Baseline srcnn

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Add, Conv2D, Input, Lambda, UpSampling2D
from tensorflow.keras.models import Model
import cv2
from tensorflow.keras.preprocessing.image import save_img
DIV2K_RGB_MEAN = np.array([0.4488, 0.4371, 0.4040]) * 255

def srcnn_res6():
    '''
    creates a srcnn model with residual blocks
    '''
    x_in = Input(shape=(None, None, 3))
    x = (x_in - DIV2K_RGB_MEAN) / 127.5 # normalize
    
    x = Conv2D(64 * (2 ** 2), 3, padding='same')(x) # pre-upsampling
    x = tf.nn.depth_to_space(x, 2) 
    x = Conv2D(64 * (2 ** 2), 3, padding='same')(x)
    x = tf.nn.depth_to_space(x, 2)   
    
    b = Conv2D(64,3,padding = 'same', activation='relu')(x)
    x = b
    
    for i in range(6):
        x_b = Conv2D(64, 3, padding='same', activation='relu')(b)
        x_b = Conv2D(64, 3, padding='same', activation='relu')(x_b)
        b = Add()([x_b, b])        
    
    x = Add()([x, b])
    
    x = Conv2D(3, 3, padding='same')(x)
    
    x = x * 127.5 + DIV2K_RGB_MEAN # denormalize
    return Model(x_in, x)


In [23]:
model_res6 = srcnn_res6()

In [24]:
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.python.keras.models import Model

# Adam optimizer with a scheduler that halfs learning rate after 20,000 steps
optim_srcnn = Adam(learning_rate=PiecewiseConstantDecay(boundaries=[20000], values=[1e-4, 5e-5]))

# Compile and train model for 300,000 steps with L1 pixel loss
model_res6.compile(optimizer=optim_srcnn, loss='mean_squared_error')
history_res6 = model_res6.fit(train_ds, epochs=50, steps_per_epoch=1000)

# Save model weights
weights_dir = 'weights/'
model_res6.save_weights(os.path.join(weights_dir, 'weights-srcnn-res6-mse-x4.h5'))

Epoch 1/5
100/100 [==============================] - 15s 149ms/step - loss: 1146.4373
Epoch 2/5
100/100 [==============================] - 15s 147ms/step - loss: 344.6303
Epoch 3/5
100/100 [==============================] - 15s 148ms/step - loss: 265.0926
Epoch 4/5
100/100 [==============================] - 15s 150ms/step - loss: 246.6658
Epoch 5/5
100/100 [==============================] - 15s 149ms/step - loss: 232.9064


# Deep SRCNN (inception block)

In [9]:
from data import DIV2K
train = DIV2K(scale=4, downgrade='bicubic', subset='train')
train_ds = train.dataset(batch_size=16, random_transform=True)

In [10]:
# Baseline srcnn
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Add, Conv2D, Input, Lambda, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
import cv2
from tensorflow.keras.preprocessing.image import save_img
DIV2K_RGB_MEAN = np.array([0.4488, 0.4371, 0.4040]) * 255

def srcnn_incep3():
    '''
    creates a srcnn model with residual blocks
    '''
    x_in = Input(shape=(None, None, 3))
    x = (x_in - DIV2K_RGB_MEAN) / 127.5 # normalize
    
    x = Conv2D(64 * (2 ** 2), 3, padding='same')(x) # pre-upsampling
    x = tf.nn.depth_to_space(x, 2) 
    x = Conv2D(64 * (2 ** 2), 3, padding='same')(x)
    x = tf.nn.depth_to_space(x, 2)   
    
    for i in range(3):
        x1 = Conv2D(16, 1, padding='same', activation='relu')(x)
        x2 = Conv2D(16, 3, padding='same', activation='relu')(x)
        x3 = Conv2D(16, 5, padding='same', activation='relu')(x)
        x4 = Conv2D(16, 9, padding='same', activation='relu')(x)
        x = Concatenate(axis = -1)([x1,x2,x3,x4])
        
    x = Conv2D(3, 5, padding='same')(x)
    
    x = x * 127.5 + DIV2K_RGB_MEAN # denormalize
    return Model(x_in, x)


In [11]:
model_incep3 = srcnn_incep3()

In [12]:
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.python.keras.models import Model

# Adam optimizer with a scheduler that halfs learning rate after 20,000 steps
optim_srcnn = Adam(learning_rate=PiecewiseConstantDecay(boundaries=[20000], values=[1e-4, 5e-5]))

# Compile and train model for 300,000 steps with L1 pixel loss
model_incep3.compile(optimizer=optim_srcnn, loss='mean_squared_error')
history_incep3 = model_incep3.fit(train_ds, epochs=50, steps_per_epoch=1000)

# Save model weights
weights_dir = 'weights/'
model_incep3.save_weights(os.path.join(weights_dir, 'weights-srcnn-incep3-mse-x4.h5'))

Epoch 1/5
100/100 [==============================] - 41s 415ms/step - loss: 1662.8253
Epoch 2/5
100/100 [==============================] - 41s 410ms/step - loss: 568.1018
Epoch 3/5
100/100 [==============================] - 41s 411ms/step - loss: 385.2892
Epoch 4/5
100/100 [==============================] - 35s 353ms/step - loss: 316.6289
Epoch 5/5
100/100 [==============================] - 41s 410ms/step - loss: 281.8763
